In [10]:
"""

CVP/LLD DEC Web Scraper DRG activities
Created September 202s
@author: Rick Morgan (Derived from worked produced by Yehuda Magid)


"""
import pandas as pd
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import requests
import re
from retry import retry
import pickle
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException      
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys #for additional keys like down arrow
import ast
from tqdm import tqdm_notebook as tqdm #Loop wrapper for progress bar
import os
import itertools

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # print all cell outputs not just last
pd.set_option('display.max_colwidth', None) # Do not cut off column values

pd.options.display.max_rows = 4000
pd.options.display.max_columns = None

#------------------------------------------------------------------------------
# Set path where your chrome driver is located (needed to run Selenium library)

chromeDriverPath = r"C:\Users\rickm\chromedriver.exe"

'\n\nCVP/LLD DEC Web Scraper DRG activities\nCreated September 202s\n@author: Rick Morgan (Derived from worked produced by Yehuda Magid)\n\n\n'

In [11]:
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

#driver = webdriver.Chrome(executable_path = chromeDriverPath) 
#driver.get('https://dec.usaid.gov/dec/content/AdvancedSearch.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy')

In [12]:
"""
os.chdir("data")
cwd = os.getcwd()
#cwd = os.chdir(r"data")
print('New working directory:', cwd)

"""

'\nos.chdir("data")\ncwd = os.getcwd()\n#cwd = os.chdir(r"data")\nprint(\'New working directory:\', cwd)\n\n'

In [13]:

#------------------------------------------------------------------------------
# Set Parameter variables

#searchTermsList = ['CVE', 'Early Warning']
#searchType = 'Title'

searchTermsList = []
 
docTypeList = ['Final Evaluation Report',
               'Special Evaluation', 
              'Other USAID Evaluation', 
              'Final Contractor/Grantee Report']

primarySubjectsList = []

geographicTermsList = []

languageList = ['English']

yearList = '2017 OR 2018 OR 2019 OR 2020 OR 2021 OR 2022'

#searchTermsList = ['CVE', 'climate AND conflict']"""

In [14]:
#------------------------------------------------------------------------------
# Define Functions:

def searchTermsFuncYears(yearList, driver):
    # Enter search terms into the doc search box if there are search terms
    if yearList:
    # Enter search terms into the search box
       #search_box = driver.find_element_by_id("ctl00_ctl00_cntrlID_51543")
        search_box = driver.find_element(by=By.ID, value="ctl00_ctl00_cntrlID_51543")
        search_box.send_keys(yearList)
        
def pressEnter(driver):
    # Click search button to run search
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to bottom of page
    button = driver.find_element_by_id("ctl00_ctl00_btnSubmit")
    ActionChains(driver).move_to_element(button).click(button).perform()
    
def enterDocType(docType, driver):
    # Function selects all document types listed in the docType list
    # defined by the user
    # Assign the document type text box to variable "box"
    box = driver.find_element_by_id("ctl00_ctl00_cntrlID_51529_srcList")
    #get all the options into a list
    options = box.find_elements_by_tag_name("option") 
    # Assign the button to add selected document type to variable "add button"
    addButton = driver.find_element_by_id("ctl00_ctl00_cntrlID_51529_add")
    
    # Create empty list that you will add the document type options available in the text box to
    optionsList = []
    for option in options: #iterate over the options, place attribute value in list
        optionsList.append(option.get_attribute("value"))
    
    # For each option, select it and add to search if it is in the user provided 
    # document type list
    for optionValue in optionsList:
        if optionValue in docType:
            select = Select(driver.find_element_by_id("ctl00_ctl00_cntrlID_51529_srcList"))
            select.select_by_value(optionValue)
            addButton.click() #
            
def enterPrimarySubects(primarySubjects, driver):
    # Function selects all primary subjects listed in the PrimarySubjects list
    # defined by the user
    
    # Assign the document type text box to variable "box"
    box = driver.find_element_by_id("ctl00_ctl00_cntrlID_52914_srcList")
    #get all the options into a list
    options = box.find_elements_by_tag_name("option") 
    # Assign the button to add selected document type to variable "add button"
    addButton = driver.find_element_by_id("ctl00_ctl00_cntrlID_52914_add")
    
    # Create empty list that you will add the document type options available in the text box to
    optionsList = []
    for option in options: #iterate over the options, place attribute value in list
        optionsList.append(option.get_attribute("value"))
    
    # For each option, select it and add to search if it is in the user provided 
    # document type list
    for optionValue in optionsList:
        if optionValue in primarySubjects:
            select = Select(driver.find_element_by_id("ctl00_ctl00_cntrlID_52914_srcList"))
            select.select_by_value(optionValue)
            addButton.click() #                

def enterLanguages(languages, driver):
    # Function selects all primary subjects listed in the PrimarySubjects list
    # defined by the user
    
    # Assign the document type text box to variable "box"
    box = driver.find_element_by_id("ctl00_ctl00_cntrlID_51575_srcList")
    #get all the options into a list
    options = box.find_elements_by_tag_name("option") 
    # Assign the button to add selected document type to variable "add button"
    addButton = driver.find_element_by_id("ctl00_ctl00_cntrlID_51575_add")
    
    # Create empty list that you will add the document type options available in the text box to
    optionsList = []
    for option in options: #iterate over the options, place attribute value in list
        optionsList.append(option.get_attribute("value"))
    
    # For each option, select it and add to search if it is in the user provided 
    # document type list
    for optionValue in optionsList:
        if optionValue in languages:
            select = Select(driver.find_element_by_id("ctl00_ctl00_cntrlID_51575_srcList"))
            select.select_by_value(optionValue)
            addButton.click() #
            
def switchToLatestWindow():
    #Make last window opened the active window
    window2 = driver.window_handles[-1]
    driver.switch_to.window(window2)

def scrapeTable(driver):
    #Scrape table from the more details page and format it into a wide dataframe
    ## assign table element to variable tbl
    tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
    ## convert html to pandas
    df  = pd.read_html(tbl)
    ## Pull out the first table in the list of tables
    df = df[0]
    ## Keep only the first two columns
    df = df.iloc[: , :2]
    ## Transpose the dataframe
    df = df.T
    ## Assign the first row to column names
    df.columns = df.iloc[0]
    ## Drop the first row that has been assigned to column names
    df =  df.iloc[pd.RangeIndex(len(df)).drop(0)]
    ## Return the cleaned dataframe
    return df

# Click Documents button to filter for documents
@retry(delay=3, jitter=2, max_delay=6, tries=4)
def clickDocType(driver):
    #driver.find_element_by_xpath('//*[@id="DocumentTypes"]').send_keys("\n") # click excel to select document file type
    button = driver.find_element_by_xpath('//*[@id="DocumentTypes"]')
    driver.implicitly_wait(10)
    ActionChains(driver).move_to_element(button).click(button).perform()
    sleep(1)
    
def clickNext(driver):
    #Click button to move to next page of results
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to bottom of page
    nextButton = driver.find_element_by_xpath('//*[@title="Next Page"]')
    driver.implicitly_wait(10)
    ActionChains(driver).move_to_element(nextButton).click(nextButton).perform()
    sleep(2)
    
def checkNext(driver):
    #Check whether there are any additional pages of results
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to bottom of page
    button = driver.find_element_by_xpath('//*[@title="Next Page"]')
    disabled = button.get_property('disabled')
    if disabled == False:
        lastPage = False
    else:
        lastPage = True
        
    return lastPage
    
def checkNoResults(driver):
    # Function checks if the search returned no results
    sleep(3)
    header = driver.find_element_by_class_name('search-summary-header-text').text
    resultCheck = 'No matches were found'
    if resultCheck in header:
        noResults = True
    else:
        noResults = False
    
    return noResults

def GetMoreDetailsLinks(driver):
    # Function scrapes all of the web links for the individual resources
    # on the dec advanced search results page
    links = driver.find_elements_by_class_name('record-detail-link')    
    hrefs=[]
    for i, link in enumerate(links):
        try:
            hrefs.append(link.get_attribute("href"))
        except (StaleElementReferenceException):
            print("Could not access more info for the resource at the following link:", link )
    if not hrefs:
        print("Help!!! The browser window is stuck and needs you to click NEXT!!!")
    return hrefs

@retry(delay=3, jitter=2, max_delay=6, tries=4) #set rety criteria
def GetMoreDetailsLinksFull(driver):
    # Function gets all links to the more details page for all results
    hrefs=[]
    morePages = True
    while morePages == True:   
        href = GetMoreDetailsLinks()
        clickNext(driver)
        try: 
            hrefs.append(href)
        except (StaleElementReferenceException):
            print("Stale reference, could not be added to list of refrences")
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to bottom of page
        nextExists = checkNext('//*[@title="Next Page"]') # make sure there is a next option (i.e. not the last page)
    
        if nextExists == False:
            morePages = False
    sleep(randint(.5,1))
    
    return hrefs

@retry(delay=3, jitter=2, max_delay=6, tries=4) #set rety criteria
def getAllMoreDeatailsLinks(driver):
    linksFull = []
    lastPage = False
    noResults = False
    while (lastPage == False) and (noResults == False):
        sleep(2)
        # Check if the search returned no results
        noResults = checkNoResults(driver)
        if noResults == False:
            # Get a list of links to the "more details" page for each resource
            links = GetMoreDetailsLinks(driver)
            linksFull.append(links)
            # Check if the search returned no results
            # Check if there is a working next page link (i.e. you are not on last page)
            lastPage = checkNext(driver)
            # click next to move to next page of results page
            if lastPage == False:
                clickNext(driver)
    # Create a list of links with no duplicates
    linksFullFlat = [item for sublist in linksFull for item in sublist]
    linksFullFlat = set(linksFullFlat)
    print(len(linksFullFlat), "Links have been scraped")
    return linksFullFlat

@retry(delay=3, jitter=2, max_delay=6, tries=4) #set rety criteria
def resourceToTable(url, driver):
    # Function transforms information on an individual resource page to a dataframe
    df = scrapeTable(driver)
    df = df.loc[:, df.columns.notnull()]
    df.drop(list(df.filter(regex = '^False')), axis = 1, inplace = True)
    
    return df

def createTableOfReferences(linksFull, driver):
    # Function loops through all resource URLs, collects the resource information,
    ## and creates a single dataframe with a row for each resource
    resources = pd.DataFrame()
    missingInfo = 1
    for i, link in enumerate(tqdm(linksFull)):
        try:
            url = driver.get(link)
        except:
            missingInfo += 1
            pass
        try:
            resource = resourceToTable(link, driver)
            resources = resources.append(resource) #Append each resource's info to the master resource df
        except (NoSuchElementException):
            missingInfo += 1
            pass
    # Print number of resources that were dropped due to errors
    print(missingInfo, "resources were dropped due to errors")
    
    return resources


def main(docType='All', years='All', 
         primarySubjects = 'All', geographicTerms='All', languages='All'):
    # This is the main function that runs the entire program
    #Open Driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    #driver = webdriver.Chrome(executable_path = chromeDriverPath)
    sleep(1)
    resourceLinks = []
        #Go to URL for advanced DEC search (update the link if it has changed)
    driver.get('https://dec.usaid.gov/dec/content/AdvancedSearch.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY2Uy')
        # Select document types to be included in the search results
    if docTypeList != 'All':
        enterDocType(docType, driver)
        # Add year filter to the search
    if years != 'All':
        searchTermsFuncYears(yearList, driver)
        # Add primary subjects filter to the search
    if primarySubjects != 'All':
        enterPrimarySubects(primarySubjectsList, driver)
        # Add geographic terms to the search
    if geographicTerms != 'All':
        enterGeographicTerms(geographicTermsList, driver)
        # Add language filter to the search
    if languages != 'All':
        enterLanguages(languageList, driver)
        # Add search terms to either titles or resource texts
    
    driver.maximize_window()
        # click button to run search
    pressEnter(driver)
    sleep(5)
        # Get all URLs to the more detail pages for references returned in search
    linksFullFlat = getAllMoreDeatailsLinks(driver)
    resourceLinks.append(linksFullFlat)
    # Convert the list of sets into a single flat list
    resourceLinksFullFlat = list(set(list(itertools.chain.from_iterable(resourceLinks))))
    # Save list of resource links as pickle file
    with open('List of Resource Links in DEC.pkl', 'wb') as f:
        pickle.dump(linksFullFlat, f)
    # create dataframe of resource information found in the more links pages
    resourcesDF = createTableOfReferences(resourceLinksFullFlat, driver)
    # Save resource data frame as a pickle file
    with open('DEC Resources Dataframe.pkl', 'wb') as f:
        pickle.dump(resourcesDF, f)
    # Close driver
    driver.quit()
    
    return resourcesDF

#------------------------------------------------------------------------------
# Run program:
          
resourcesDF = main(docType=docTypeList, years=yearList, 
                   primarySubjects='All', geographicTerms='All', languages='All')

#------------------------------------------------------------------------------
# Export results to excel
resourcesDF.to_excel('Evaluation Final report docType scraped SEP 2022.xlsx', sheet_name='Search Results', 
                     index=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\rickm\.wdm\drivers\chromedriver\win32\105.0.5195.52]
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  box = driver.find_element_by_id("ctl00_ctl00_cntrlID_51529_srcList")
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:24: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  options = box.find_elements_by_tag_name("option")
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Pleas

3286 Links have been scraped


C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:229: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, link in enumerate(tqdm(linksFull)):


  0%|          | 0/3286 [00:00<?, ?it/s]

C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resources = resources.append(resource) #Append each resource's info to the master resource df
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local

...

C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resources = resources.append(resource) #Append each resource's info to the master resource df
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local

C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resources = resources.append(resource) #Append each resource's info to the master resource df
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local

C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resources = resources.append(resource) #Append each resource's info to the master resource df
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local

C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resources = resources.append(resource) #Append each resource's info to the master resource df
C:\Users\rickm\AppData\Local\Temp\ipykernel_11304\2506918278.py:97: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  tbl = driver.find_element_by_xpath("/html/body/div[1]/div[4]/form/div[4]/div[2]/div/div[2]/div/div/div/div[2]/center/table").get_attribute('outerHTML')
C:\Users\rickm\AppData\Local

1 resources were dropped due to errors
